In [2]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid


# Class

In [12]:
# define class

class Reviews:
    def __init__(self, id, item_id, review_time, variant, content):
        self.id = id
        self.item_id=item_id
        self.review_time = review_time
        self.variant = variant
        self.content = content

class Items:
    def __init__(self, id, name, ratings, ratings_num, sold, shop_name, url):
        self.id=id
        self.name=name
        self.ratings=ratings
        self.ratings_num=ratings_num
        self.sold=sold
        self.shop_name=shop_name
        self.url=url


# Main File

### Open Chrome

In [5]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
# Make sure you have the appropriate version of ChromeDriver for your installed Chrome browser
chrome_driver_path = 'D:/OneDrive/OneDrive - Asia Pacific University/FYP/Scraping Practice/Chrome Driver'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL - men's wallet category
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_17988\1395372814.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


### Extract Link Using Beautiful Soup

In [224]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

/Shoe-Deodorant-Spray-Men-Women-260ml-Shoe-Spray-Deodorant-鞋子除臭喷雾-Pewangi-Kasut【Ready-Stock】Shoes-Deodorant-Spray-i.602913319.18126897326?sp_atk=723bc4d4-9661-4689-9b2b-6f4f7f2876b5&xptdk=723bc4d4-9661-4689-9b2b-6f4f7f2876b5
/M.CLASS-Men-Crocs-Duet-Sport-Clogs-Mules-Sandal-Shoes-Kasut-Sarung-Lelaki-i.129247830.3358066945?sp_atk=3466dcd3-ebe8-4833-a4bd-892c2bef544e&xptdk=3466dcd3-ebe8-4833-a4bd-892c2bef544e
/-Stok-Sedia-Unisex-BALENC1AGA-Letter-sandal-Perempuan-Lelaki-Kasut-Fesyen-i.241427433.14795346094?sp_atk=9d942450-cbf9-40c0-b395-0e204eefc753&xptdk=9d942450-cbf9-40c0-b395-0e204eefc753
/✅-（in-stock）nike-air-force-1-classic-men-shoes-sneakers-women-casual-shoes-black-White-Red-i.402245566.11924251403?sp_atk=695289c0-6660-4da4-ba85-4890b9c601f1&xptdk=695289c0-6660-4da4-ba85-4890b9c601f1
/READY-STOCK-SNEAKERS-CLEANING-TOOLS-SHOES-CLEANING-COMBO-SET-SHOES-CARE-AND-CLEANING-TOOLS-i.74589099.4351883855?sp_atk=e79236c1-744c-4467-9984-a014fa10c4f1&xptdk=e79236c1-744c-4467-9984-a014fa10c4f1


In [225]:
# Len of the list

print(len(the_links))


60


### Get details of the products

In [226]:
item_list=[]
reviews_list=[]

In [227]:
counter=0

In [234]:

for i in range (36,len(the_links)):

    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)

    # ensure link is open
    time.sleep(5)
    soup=BeautifulSoup(browser.page_source, "html.parser")
    
    # extract details
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="P3CdcB").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)

    # record counter to know where to continue from
    print(counter)
    counter=counter+1

    # append in list
    item_list.append(Items(id,title,ratings,ratings_num,sold,shop_name,links))

    # REVIEWS
    for i in range (0,30): 
        soup=BeautifulSoup(browser.page_source, "html.parser")

        review_containers=soup.findAll('div',class_="shopee-product-rating__main")

        for rc in review_containers:
            review_id=uuid.uuid4()
            item_id=id
            time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")
            
            if(len(time_var)==2):
                review_time=time_var[0]
                variant=time_var[1]
            else:
                review_time=time_var[0]
                variant="None"
            
            reviews=rc.find('div',class_="Rk6V+3")
            
            if(reviews!=None):
                reviews=reviews.get_text(strip=True)
            else:
                reviews="None"
            reviews_list.append(Reviews(review_id, item_id, review_time,variant,reviews))

        time.sleep(1)
        WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
        time.sleep(1)



36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [235]:
df = pd.DataFrame([vars(obj) for obj in item_list])

In [236]:
df.shape

(60, 7)

In [237]:
df.head(5)

,id,name,ratings,ratings_num,sold,shop_name,url
0,77221cdb-ab25-4ca4-a3e7-922c5aeda5c5,Shoe Deodorant Spray Men Women 260ml Shoe Spra...,4.9,1.2k,7.5k,goooodmall,https://shopee.com.my/Shoe-Deodorant-Spray-Men...
1,d13a9172-b353-4ce1-96fb-8bb44c13bdda,M.CLASS Men Crocs Duet Sport Clogs Mules Sanda...,4.9,2.4k,4.9k,minmin_yong,https://shopee.com.my/M.CLASS-Men-Crocs-Duet-S...
2,7ae2dc27-c6e7-49c3-9a6d-bd50bebb6bb8,[Stok Sedia] Unisex BALENC1AGA Letter sandal P...,4.8,175,503,vg.collection_store,https://shopee.com.my/-Stok-Sedia-Unisex-BALEN...
3,12702d16-48e4-46f9-b04e-a510a3e92f51,✅ （in stock）nike air force 1 classic men shoe...,4.9,2k,5.1k,skynew4898,https://shopee.com.my/✅-（in-stock）nike-air-for...
4,31100126-c99e-4307-8a8b-b85fce784ece,READY STOCK SNEAKERS CLEANING TOOLS SHOES CLEA...,4.9,1.6k,3.4k,joeyap2828,https://shopee.com.my/READY-STOCK-SNEAKERS-CLE...


In [238]:
df2 = pd.DataFrame([vars(obj) for obj in reviews_list])

In [239]:
df2.shape
#df2.head(10)

(7200, 5)

In [215]:
df.to_csv('items11.csv', encoding='utf-8', index=False)
df2.to_csv('review11.csv', encoding='utf-8', index=False)

# Part 2

Extract description and price

In [3]:
df=pd.read_csv("items6.csv")

In [6]:
import numpy as np

# Get the content of the column into a NumPy array
column_array = df['url'].values

# Print the array
print(column_array)

["https://shopee.com.my/TSCfashion-Large-Capacity-Tote-Bag-Women's-New-Fashion-Corduroy-Bucket-Bag-Student-Class-Shoulder-Bag-Sling-Bag-i.570000833.15756682997?sp_atk=31376d9d-6f2e-4e32-aad9-40c427fc25dc&xptdk=31376d9d-6f2e-4e32-aad9-40c427fc25dc"
 'https://shopee.com.my/Japanese-and-Korean-Cute-Backpack-Female-Student-Cartoon-Schoolbag-New-Graffiti-Travel-Bag-i.279532416.20968753343?sp_atk=02a6f589-6b96-4c45-b177-0b16eada93f2&xptdk=02a6f589-6b96-4c45-b177-0b16eada93f2'
 'https://shopee.com.my/(11-Colors)💕-Multi-Function-Travel-Storage-Bag-Cute-Pattern-Large-Capacity-Sanitary-pad-pouch-Girls-Physiological-Period-Tampon-Organiser-Bag-卫生巾收纳-i.74757901.21201686938?sp_atk=26b3de43-d850-4397-90dd-f63471da3996&xptdk=26b3de43-d850-4397-90dd-f63471da3996'
 'https://shopee.com.my/LANFY-Fashion-Fluffy-Shoulder-Bag-Female-Top-handle-Bag-Shopping-Bag-Autumn-Winter-Plush-Tote-Women-Girls-Handbag-i.50713577.9294714333?sp_atk=e9e39bab-41bc-4542-81d2-4991a8480232&xptdk=e9e39bab-41bc-4542-81d2-4991a848

In [7]:

# Create empty columns for description and price
df['description'] = ""
df['price'] = ""

for index, row in df.iterrows():
    url = row['url']
    
    # Open link
    browser.get(url)
    time.sleep(5)
    
    # Extract details using BeautifulSoup
    soup = BeautifulSoup(browser.page_source, "html.parser")
    descriptions = soup.find_all('p', class_='irIKAp')
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    print(text)
    #print(price)
    # Update dataframe with description and price
    df.at[index, 'description'] = text
    df.at[index, 'price'] = price

  


👑 Welcome to my shop
🚀 All products have been carefully checked before delivery, please feel free to buy
🌈 Product description:

Material: nylon
Whether waterproof: waterproof
Opening method: zipper
Length, width and height: 30 * 13 * 42CM
Color: Light pink, light green, light purple, light blue, black
Interior space: multiple compartments
Computer size:15.6 inches
Cubicle with or without computer: Yes

🌞 If you have any questions or concerns, please contact our customer service personnel, and we will provide the best service
🌠 Enjoy your shoppi
💕Welcome to【SOHOME.MY】
💕Please remember to Follow our shop and receive store coupons before placing an order~
💕Follow us to enjoy priority delivery and get the latest discounts
Type: Storage Bag
Size:
#1, SIZE S, 12cm x 12cm x 1.5cm
#2, SIZE M, 13cm x 13.5cm x 4cm
💕Description
Cute large-capacity sanitary napkin storage bag, a cartoon girl’s heart-menstrual sanitary napkin portable storage bag. Large-capacity, multi-purpose, can store cosmetic

In [8]:
df.head(60)

,id,name,ratings,ratings_num,sold,shop_name,url,description,price
0,c45db2cd-ab2a-4f38-be55-a29e528d7aaf,TSCfashion Large Capacity Tote Bag Women's New...,4.8,1.4k,3.7k,tscfashion.my,https://shopee.com.my/TSCfashion-Large-Capacit...,,RM10.50
1,34426087-982f-4f79-be1b-8f0dd833d72a,Japanese and Korean Cute Backpack Female Stude...,4.9,340,791,lifefine.my,https://shopee.com.my/Japanese-and-Korean-Cute...,👑 Welcome to my shop\n🚀 All products have been...,RM24.80 - RM26.80
2,5438d0e1-415c-47e7-83a5-1d721e3db31d,(11 Colors)💕 Multi-Function Travel Storage Bag...,4.8,765,3k,daylan2.my,https://shopee.com.my/(11-Colors)💕-Multi-Funct...,💕Welcome to【SOHOME.MY】\n💕Please remember to Fo...,RM2.80 - RM4.86
3,db943c9c-1a03-4ea0-ae61-ec30c8981f41,LANFY Fashion Fluffy Shoulder Bag Female Top-h...,4.8,1.2k,3.9k,lanfygoods.my,https://shopee.com.my/LANFY-Fashion-Fluffy-Sho...,------------------Welcome to Lanfygoods store-...,RM8.03
4,a4a355f7-e23e-4164-ad54-de046bfde566,7-color Felt tote Bag Strawberry Bear Bag High...,4.8,735,5.1k,charian.my,https://shopee.com.my/7-color-Felt-tote-Bag-St...,"{""field_list"":[{""type"":0,""value"":""🥳🥳Welcome To...",RM1.88 - RM9.88
5,b086097e-40d3-4ce7-87af-cb1864f46b0b,N323 READY STOCK MYFOOYIN 7 COLORS Felt bag ca...,4.9,732,5.7k,fooyin88,https://shopee.com.my/N323-READY-STOCK-MYFOOYI...,READY STOCK\nMALAYSIA\nSIZE: LONG 30CM X HEIGH...,RM6.30
6,7ac7af7c-260d-4bde-a90e-a428651a2b87,EverToner Felt Insert Bag Fits For LongChamp H...,4.9,1.5k,4.6k,evertoner.my,https://shopee.com.my/EverToner-Felt-Insert-Ba...,Product Name:Felt Liner Bag for Longchamp\nCol...,RM13.50 - RM15.90
7,27a9c227-acb4-4e9e-8f82-de646d117fa3,Chaika Kilter Women's Bag for Saddle Shoulder ...,4.9,616,1.6k,chaikakilter.os,https://shopee.com.my/Chaika-Kilter-Women's-Ba...,✨Welcome to Chaika Kilter\n❤Provide profession...,RM19.45
8,761b55e5-0810-48d8-8eaa-b7e29cd645eb,(READY STOCK) 12oz Plain Canvas Bag Plain Tote...,4.9,235,9.6k,rasa.sayang,https://shopee.com.my/(READY-STOCK)-12oz-Plain...,🌈Welcome to my shop- RASA.SAYANG\n\n✅ We are ...,RM3.80 - RM6.00
9,d20c19ed-ff93-46a7-b2c5-e68f24b426f9,Portable Earphones Storage Bag USB Data Cable ...,5.0,616,3.1k,mayleebiz,https://shopee.com.my/Portable-Earphones-Stora...,Product Content\n\nReady Stock in Malaysia.\nS...,RM2.59 - RM5.19


In [9]:
df.to_csv('items6.csv', index=False)

In [10]:
df.to_json('items6.json', orient='records')

In [16]:

df.head()

,id,name,ratings,ratings_num,sold,shop_name,url,description,price
0,c68cc4e7-4780-4a0a-8df3-05931e3fecbe,MALAYSIA STOCK COSCO B117 Unisex School Bag Ba...,4.8,879,2.4k,coscopremium,https://shopee.com.my/MALAYSIA-STOCK-COSCO-B11...,(Without Keychain)\n\n**30CM x 38CM x 10CM\n\n...,RM4.90 - RM17.90
1,fd645e16-5837-4367-9cda-7f8576ed5076,Hadiah Set!!! Dompet Kulit CK Wallet Leather G...,4.8,2.3k,4.7k,ihousehold,https://shopee.com.my/Hadiah-Set!!!-Dompet-Kul...,📦READY STOCK Di Malaysia \nIHouseHold.co \n\n🌟...,RM59.90
2,625d42b3-e129-4d2c-ac6b-b03ec5a79e64,LABER LEE Men Shoulder Canvas Crossbody Bag,4.9,3.4k,6.4k,csonlinemall.os,https://shopee.com.my/LABER-LEE-Men-Shoulder-C...,💕 Welcome to LABER LEE 💕\n💯 READY STOCK IN MAL...,RM17.90
3,244dad3f-1130-4150-80c5-c90f3a145fb7,Dr Cardin Carter II Men Waist Pouch Travel Bag...,4.9,7.3k,12.1k,drcardin.os,https://shopee.com.my/Dr-Cardin-Carter-II-Men-...,Dr Cardin has been manufacturing and distribut...,RM49.00
4,2e9f5921-db6c-4003-80a3-afa66a17f4fa,*Ready Stock* School Bags for Student Boys Gir...,4.9,992,2.2k,h.s.h_trading,https://shopee.com.my/*Ready-Stock*-School-Bag...,\n\n\n\n\n**** BOTH DESIGNS SAME SIZE **** ***...,RM24.99


Extract pic

In [3]:
df=pd.read_json("all_products.json")

In [4]:
import numpy as np

# Get the content of the column into a NumPy array
column_array = df['url'].values

# Print the array
print(column_array)

['https://shopee.com.my/NIKE-Fashion-Travel-School-Backpack-Bag-For-Unisex-i.7533099.18027251861?sp_atk=ac0842c7-3c7f-4cdf-a31c-4f98564f75d6&xptdk=ac0842c7-3c7f-4cdf-a31c-4f98564f75d6'
 'https://shopee.com.my/Wallet-For-Men-PU-Leather-Card-Holder-Wallet-Rfid-Mini-Slim-Wallet-Dompet-Kulit-Lembu-Lelaki-Aluminum-Auto-POP-Up-Birthday-Gift-For-Men-i.742492399.17445086619?sp_atk=44aa2500-6c49-440f-a2e1-d766c00a8d72&xptdk=44aa2500-6c49-440f-a2e1-d766c00a8d72'
 'https://shopee.com.my/READY-STOCK-❤️-Premium-9-14-Inch-Laptop-Bag-Pouch-Small-Size-Shockproof-Tablet-Sleeve-Labtop-Bag-Sleeves-i.120853067.16541348284?sp_atk=d7c72dd1-bffa-4fb4-8142-c73136c5c266&xptdk=d7c72dd1-bffa-4fb4-8142-c73136c5c266'
 'https://shopee.com.my/CS-New-Men-Leather-Messenger-Bag-Single-Shoulder-Bag-Business-Casual-Crossbody-Bag-i.118061149.1811232519?sp_atk=efa68b5a-de37-435e-b465-20978fd4f56f&xptdk=efa68b5a-de37-435e-b465-20978fd4f56f'
 'https://shopee.com.my/Ready-Stock-Nike-Backpack-Bag-Nike-Unisex-Air-Laptop-Sport-T

In [7]:
# Create empty columns for pic
df['image']=""

for index, row in df.iterrows():
    url = row['url']
    
    # Open link
    browser.get(url)
    time.sleep(5)
    
    # Extract details using BeautifulSoup
    soup = BeautifulSoup(browser.page_source, "html.parser")
    div_element = soup.find('div', class_='A4dsoy aehCaY')

    if div_element:
        style_attr = div_element.get('style')
        image = None

    if style_attr:
        image_start = style_attr.find('url("') + 5
        image_end = style_attr.find('")', image_start)

        if image_start != -1 and image_end != -1:
            image = style_attr[image_start:image_end]

    print(image)
    # Update dataframe with description and price
    df.at[index, 'image'] = image
    

https://down-my.img.susercontent.com/file/65b270cf317f671dbaca6af15e59b733_tn
https://down-my.img.susercontent.com/file/662c301a6b381dc46ffd8e9b6cad1373_tn
https://down-my.img.susercontent.com/file/9a45e27330fd77f8cfd61a2f22b00243_tn
https://down-my.img.susercontent.com/file/1a5961d0034e152d92b5c6364636fe25_tn
https://down-my.img.susercontent.com/file/sg-11134201-22110-zl5syfvubdjv16_tn
https://down-my.img.susercontent.com/file/b523360999f38f1b9b7ed7478d69ae48_tn
https://down-my.img.susercontent.com/file/64c13bf08fe7d56ce91cc15a043e1c68_tn
https://down-my.img.susercontent.com/file/11ede2970d48063642e95b4313d342e4_tn
https://down-my.img.susercontent.com/file/7e79ce7257813f18c64e9dc26555ae37_tn
https://down-my.img.susercontent.com/file/b84aad70214461a8c91fcbcd97e1e6c2_tn
https://down-my.img.susercontent.com/file/760a567e5757032dd6de27f9f7cbeb08_tn
https://down-my.img.susercontent.com/file/024a9f995471ec5e8e866c85d9a9be2e_tn
https://down-my.img.susercontent.com/file/3066eea59597555fae825b

In [8]:
df.head()

,id,name,ratings,ratings_num,sold,shop_name,url,description,price,accu_score,image
0,20bcb1c5-1b2b-46e4-9586-db1fa0d7d371,NIKE Fashion TravelSchool Backpack Bag For Unisex,4.8,162,447,sunsuncps,https://shopee.com.my/NIKE-Fashion-Travel-Scho...,High quality Pre-order For Unisex Fashion Bag ...,RM38.00,1.061794,https://down-my.img.susercontent.com/file/65b2...
1,988bec91-c0a1-41e8-97b4-70e324843079,Wallet For Men PU Leather Card Holder Wallet R...,4.8,8100,17800,mtfashion2.my,https://shopee.com.my/Wallet-For-Men-PU-Leathe...,MTFASHION MALAYSIA READY STOCK Ship with...,RM9.96 - RM28.99,1.218403,https://down-my.img.susercontent.com/file/662c...
2,27973f6a-ddc8-4546-a4d4-5458bffef04b,READY STOCK ❤️ Premium 914 Inch Laptop Bag Pou...,4.9,789,2000,junhomedecor,https://shopee.com.my/READY-STOCK-❤️-Premium-9...,Welcome to our fun little store All products a...,RM24.40 - RM26.40,1.359126,https://down-my.img.susercontent.com/file/9a45...
3,0c117e89-4ba7-49d4-8fe3-d2c6ea0dcfee,CS New Men Leather Messenger Bag Single Should...,4.9,934,1800,chooseset.my,https://shopee.com.my/CS-New-Men-Leather-Messe...,Welcome to chooseset • Here is some FYI fr...,RM36.80,1.314783,https://down-my.img.susercontent.com/file/1a59...
4,a3dd88b3-e6b2-441d-ae19-e0014ba1d3ad,Ready Stock Nike Backpack Bag Nike Unisex Air ...,4.8,788,2000,handbagmurah.,https://shopee.com.my/Ready-Stock-Nike-Backpac...,ALL READY STOCK MALAYSIA Local Sellers Malay...,RM26.90 - RM30.90,1.354279,https://down-my.img.susercontent.com/file/sg-1...


In [10]:
df['accu_score']=df['accu_score'].astype(str)
df['ratings']=df['ratings'].astype(str)
df['ratings_num']=df['ratings_num'].astype(str)
df['sold']=df['sold'].astype(str)


In [11]:
df.to_json('all_items.json', orient='records')